In [3]:
!pip install ezdxf
import ezdxf

def define_plot(plot_width, plot_height):
    return {
        "x_min": 0,
        "y_min": 0,
        "x_max": plot_width,
        "y_max": plot_height
    }


def define_brahmasthana(plot_width, plot_height, ratio=0.3):
    bw = plot_width * ratio
    bh = plot_height * ratio
    x1 = (plot_width - bw) / 2
    y1 = (plot_height - bh) / 2
    return {
        "x_min": x1,
        "y_min": y1,
        "x_max": x1 + bw,
        "y_max": y1 + bh
    }


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 26.3 MB/s eta 0:00:00


In [4]:
DIRECTIONS_16 = [
    "E","ENE","NE","NNE",
    "N","NNW","NW","WNW",
    "W","WSW","SW","SSW",
    "S","SSE","SE","ESE"
]

def angle_to_direction(angle):
    return DIRECTIONS_16[int((angle % 360) / (360 / 16))]

VASTU_RULES = {
    "Kitchen": {
        "primary": ["SE"],
        "good": ["SSE","W","NW"],
        "avoid": ["NE","E"]
    },
    "Bedroom": {
        "primary": ["SW","W","WSW"],
        "good": ["N","NW"],
        "avoid": []
    },
    "Living": {
        "primary": [],
        "good": [],
        "avoid": []
    },
    "Pooja": {
        "primary": ["NE"],
        "good": ["W"],
        "avoid": []
    },
    "Dining": {
        "primary": ["W"],
        "good": ["E","S"],
        "avoid": []
    },
    "Toilet": {
        "primary": ["WNW","SSW"],
        "good": ["ESE"],
        "avoid": ["NE"]
    }
}

ENTRANCE_RULES = {
    "North": ["N","NNW"],
    "South": ["SSE","SE"],
    "East": ["E","ENE"],
    "West": ["WSW","SW"]
}


ROOM_SIZES = {
    "Bedroom": (3.0, 4.8, 3.6, 4.8),
    "Kitchen": (2.1, 3.0, 3.0, 3.6),
    "Living": (3.6, 4.2, 4.2, 4.8),
    "Dining": (3.0, 3.0, 2.4, 2.4),
    "Pooja": (1.5, 2.0, 1.5, 2.0),
    "Toilet": (1.2, 2.1, 1.2, 2.1)
}


In [5]:
def score_direction(dir, rule):
    if dir in rule["primary"]:
        return 1.0
    if dir in rule["good"]:
        return 0.7
    if dir in rule["avoid"]:
        return 0.0
    return 0.3

def select_zones(room):
    zones = []
    for d in DIRECTIONS_16:
        s = score_direction(d, VASTU_RULES[room])
        if s > 0:
            zones.append({"direction": d, "score": s})
    return sorted(zones, key=lambda x: x["score"], reverse=True)


def direction_region(d):
    L,M,R = (0,0.33),(0.33,0.66),(0.66,1)
    B,C,T = (0,0.33),(0.33,0.66),(0.66,1)
    return {"N":(M,T),"NE":(R,T),"E":(R,C),"SE":(R,B), "S":(M,B),"SW":(L,B),"W":(L,C),"NW":(L,T)}.get(d,(M,C))

In [6]:
def normalize_bounds(b):
    if isinstance(b, tuple):
        return {
            "x_min": b[0],
            "y_min": b[1],
            "x_max": b[2],
            "y_max": b[3],
        }
    return b

In [7]:
def rectangles_overlap(a, b):
    a = normalize_bounds(a)
    b = normalize_bounds(b)
    return not (
        a["x_max"] <= b["x_min"] or
        a["x_min"] >= b["x_max"] or
        a["y_max"] <= b["y_min"] or
        a["y_min"] >= b["y_max"]
    )

def rectangles_adjacent(a, b, tol=0.15):
    a = normalize_bounds(a)
    b = normalize_bounds(b)

    vertical_touch = ( abs(a["x_max"] - b["x_min"]) <= tol or abs(a["x_min"] - b["x_max"]) <= tol) and not ( a["y_max"] <= b["y_min"] or a["y_min"] >= b["y_max"])

    horizontal_touch = ( abs(a["y_max"] - b["y_min"]) <= tol or abs(a["y_min"] - b["y_max"]) <= tol) and not (a["x_max"] <= b["x_min"] or a["x_min"] >= b["x_max"])

    return vertical_touch or horizontal_touch


In [8]:
def place_entrance(plot, facing, width=1.2):
    x1,y1,x2,y2 = plot
    midx = (x1+x2)/2
    midy = (y1+y2)/2

    if facing == "North":
        return ("N", (midx-width/2, y2, midx+width/2, y2))
    if facing == "South":
        return ("S", (midx-width/2, y1, midx+width/2, y1))
    if facing == "East":
        return ("E", (x2, midy-width/2, x2, midy+width/2))
    if facing == "West":
        return ("W", (x1, midy-width/2, x1, midy+width/2))

def place_room(room, plot, brahma, placed):
    pw = plot[2] - plot[0]
    ph = plot[3] - plot[1]

    minw,maxw,minh,maxh = ROOM_SIZES[room]
    w,h = (minw+maxw)/2,(minh+maxh)/2

    for z in select_zones(room):
        (xr,yr) = direction_region(z["direction"])
        cx = pw*(xr[0]+xr[1])/2
        cy = ph*(yr[0]+yr[1])/2
        rect = {
            "x_min": cx - w/2,
            "y_min": cy - h/2,
            "x_max": cx + w/2,
            "y_max": cy + h/2
        }

        if rectangles_overlap(rect, brahma):
            continue

        if any(rectangles_overlap(rect, r["bounds"]) for r in placed.values()):
            continue

        if room == "Pooja" and "Toilet" in placed:
            if rectangles_adjacent(rect, placed["Toilet"]["bounds"]):
                continue

        if room == "Toilet" and "Pooja" in placed:
            if rectangles_adjacent(rect, placed["Pooja"]["bounds"]):
                continue

        return {"bounds": rect, "direction": z["direction"]}

    return None

In [9]:
def aspect_ratio_ok(bounds, min_ratio=0.5, max_ratio=2.0):
    w = bounds["x_max"] - bounds["x_min"]
    h = bounds["y_max"] - bounds["y_min"]
    if h == 0 or w == 0:
        return False
    r = w / h
    return min_ratio <= r <= max_ratio

def expand_room(room, plot, brahma, other_rooms, step=0.25):
    bounds = normalize_bounds(room["bounds"]).copy()
    plot = normalize_bounds(plot)
    brahma = normalize_bounds(brahma)

    def can_expand(new_bounds):
        if ( new_bounds["x_min"] < plot["x_min"] or new_bounds["x_max"] > plot["x_max"] or new_bounds["y_min"] < plot["y_min"] or new_bounds["y_max"] > plot["y_max"]):
            return False

        if rectangles_overlap(new_bounds, brahma):
            return False

        for r in other_rooms:
            if rectangles_overlap(new_bounds, r["bounds"]):
                return False

        return aspect_ratio_ok(new_bounds)

    expanded = True
    while expanded:
        expanded = False
        for direction in ["left", "right", "down", "up"]:
            trial = bounds.copy()
            if direction == "left": trial["x_min"] -= step
            elif direction == "right": trial["x_max"] += step
            elif direction == "down": trial["y_min"] -= step
            elif direction == "up": trial["y_max"] += step

            if can_expand(trial):
                bounds = trial
                expanded = True

    room["bounds"] = bounds
    return room

In [10]:
CORRIDOR_W = 1.0

def mid(a,b): return (a+b)/2

def anchor(room, brahma):
    room = normalize_bounds(room)
    brahma = normalize_bounds(brahma)
    cx = mid(room["x_min"], room["x_max"])
    cy = mid(room["y_min"], room["y_max"])
    if cx < brahma["x_min"]: return (room["x_max"], cy)
    if cx > brahma["x_max"]: return (room["x_min"], cy)
    if cy < brahma["y_min"]: return (cx, room["y_max"])
    return (cx, room["y_min"])

def corridor(p1,p2):
    if p1[0]==p2[0]:
        return (p1[0]-CORRIDOR_W/2,min(p1[1],p2[1]),
                p1[0]+CORRIDOR_W/2,max(p1[1],p2[1]))
    return (min(p1[0],p2[0]),p1[1]-CORRIDOR_W/2,
            max(p1[0],p2[0]),p1[1]+CORRIDOR_W/2)

def generate_corridors(rooms, brahma):
    corridors = []
    reserved = []

    for r in rooms.values():
        a = anchor(r["bounds"], brahma)
        cx = (brahma["x_min"] + brahma["x_max"]) / 2
        cy = (brahma["y_min"] + brahma["y_max"]) / 2

        if abs(a[0] - cx) > abs(a[1] - cy):
            c = corridor(a, (cx, a[1]))
        else:
            c = corridor(a, (a[0], cy))

        def corridor_illegal(c, room):
            c = normalize_bounds(c)
            r = normalize_bounds(room["bounds"])
            overlap_x = min(c["x_max"], r["x_max"]) - max(c["x_min"], r["x_min"])
            overlap_y = min(c["y_max"], r["y_max"]) - max(c["y_min"], r["y_min"])
            return overlap_x > 0.3 and overlap_y > 0.3

        if not any(corridor_illegal(c, r) for r in rooms.values()):
            corridors.append(c)

    return corridors

def door_on_shared_wall(a, b, size=0.8):
    a = normalize_bounds(a)
    b = normalize_bounds(b)

    if a["x_max"] == b["x_min"]:
        y = mid(max(a["y_min"], b["y_min"]),
                min(a["y_max"], b["y_max"]))
        return (a["x_max"], y-size/2, a["x_max"], y+size/2)

    if a["y_max"] == b["y_min"]:
        x = mid(max(a["x_min"], b["x_min"]),
                min(a["x_max"], b["x_max"]))
        return (x-size/2, a["y_max"], x+size/2, a["y_max"])

    return None

In [11]:
def export_dxf_fixed(fname,plot_bounds,brahma_bounds,rooms,corridors,entrance=None):
    doc = ezdxf.new()
    msp = doc.modelspace()

    def rect(x1, y1, x2, y2, layer):
        msp.add_lwpolyline([(x1,y1),(x2,y1),(x2,y2),(x1,y2),(x1,y1)], dxfattribs={"layer": layer, "closed": True})

    def label(text, b, h=0.35):
        cx = (b["x_min"] + b["x_max"]) / 2
        cy = (b["y_min"] + b["y_max"]) / 2
        msp.add_text(
            text,
            dxfattribs={"height": h}
        ).set_placement((cx, cy), align=ezdxf.enums.TextEntityAlignment.MIDDLE_CENTER)

    def door(x1, y1, x2, y2):
        msp.add_line((x1,y1),(x2,y2), dxfattribs={"layer":"DOOR"})


    for lname in ["PLOT","BRAHMA","ROOM","CORRIDOR","DOOR","TEXT","ENTRANCE"]:
        if lname not in doc.layers:
            doc.layers.add(lname)

    rect(*plot_bounds, "PLOT")

    rect(brahma_bounds["x_min"], brahma_bounds["y_min"], brahma_bounds["x_max"], brahma_bounds["y_max"], "BRAHMA")

    for name, r in rooms.items():
        b = r["bounds"]
        rect(b["x_min"], b["y_min"], b["x_max"], b["y_max"], "ROOM")
        label(name, b)

    for c in corridors:
        rect(c[0], c[1], c[2], c[3], "CORRIDOR")

    for r in rooms.values():
        b = r["bounds"]
        cx = (b["x_min"] + b["x_max"]) / 2
        cy = (b["y_min"] + b["y_max"]) / 2
        door_len = 0.9
        if cx < brahma_bounds["x_min"]:
            door(b["x_max"], cy-door_len/2, b["x_max"], cy+door_len/2)
        elif cx > brahma_bounds["x_max"]:
            door(b["x_min"], cy-door_len/2, b["x_min"], cy+door_len/2)
        elif cy < brahma_bounds["y_min"]:
            door(cx-door_len/2, b["y_max"], cx+door_len/2, b["y_max"])
        else:
            door(cx-door_len/2, b["y_min"], cx+door_len/2, b["y_min"])

    if entrance:
      side = entrance["side"]
      pos = entrance["pos"]
      w = 1.2
      inset = 0.25
      if side == "N":
          door(pos-w/2, plot_bounds[3]-inset, pos+w/2, plot_bounds[3]-inset)
      elif side == "S":
          door(pos-w/2, plot_bounds[1]+inset, pos+w/2, plot_bounds[1]+inset)
      elif side == "E":
          door(plot_bounds[2]-inset, pos-w/2, plot_bounds[2]-inset, pos+w/2)
      elif side == "W":
          door(plot_bounds[0]+inset, pos-w/2, plot_bounds[0]+inset, pos+w/2)

    doc.saveas(fname)

In [12]:
if __name__ == "__main__":

    PLOT_SIZE = 20
    PLOT_FACING = "East"

    plot_dict = define_plot(PLOT_SIZE, PLOT_SIZE)
    plot_for_drawing = (0, 0, PLOT_SIZE, PLOT_SIZE)
    plot_for_logic = plot_dict

    brahma_dict = define_brahmasthana(PLOT_SIZE, PLOT_SIZE)

    entrance_side, entrance_rect = place_entrance( plot_for_drawing, PLOT_FACING)

    if entrance_side in ("N", "S"):
        entrance = {
            "side": entrance_side,
            "pos": (entrance_rect[0] + entrance_rect[2]) / 2
        }
    else:
        entrance = {
            "side": entrance_side,
            "pos": (entrance_rect[1] + entrance_rect[3]) / 2
        }

    placed = {}
    for room_name in ROOM_SIZES:
        placed[room_name] = place_room(room_name, plot_for_drawing, brahma_dict, placed)

    for name in placed:
        others = [placed[r] for r in placed if r != name]
        placed[name] = expand_room(placed[name], plot_for_logic, brahma_dict, others)

    corridors = generate_corridors(placed, brahma_dict)

    export_dxf_fixed("vastu_layout.dxf", plot_for_drawing, brahma_dict, placed, corridors, entrance=entrance)
